In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import librosa
import fileio
import keys
import full_model
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
# To create genre-specific npz files (required for randomizing splits)
#fileio.process_data_into_np_files()

# To create 27 random splits, each of size 32. (Except the last one, data/working/splits/split_26.npz is only size 5)
#fileio.create_random_splits() # Randomized with random.seed(1), so our splits will be identical

# To load a range of splits
#X, Y = fileio.load_splits(range(5)) # loads splits 0-4
#X.shape = (fileio.NUM_SAMPLES_GTZAN, 5 * 32)
#Y.shape = (24, 5 * 32)

# To generate chroma, aligned with the splits
#fileio.generate_chroma_splits_from_splits()

# To load a range of chroma splits
#X, Y = fileio.load_chroma_splits(range(5)) # loads chroma splits 0-4
#X.shape = (300, 12, 5 * 32)
#Y.shape = (24, 5 * 32)

In [3]:
def get_scores(targets, guesses):
    scores = []
    
    for idx in range(len(targets)):
        scores.append(get_score_single(targets[idx], guesses[idx, :]))
        
    return scores
    

In [4]:
def get_score_single(target, guess):
    scores = keys.get_vector_from_key(target) * 2
    return scores[keys.generate_one_hot_guess(guess.data)]

In [5]:
import importlib

importlib.reload(full_model)

net = full_model.Net()
print(net)

X, Y = fileio.load_chroma_splits(range(20))
X = torch.transpose(torch.from_numpy(X), 0, 2)
Y = torch.from_numpy(keys.generate_one_hot_matrix(Y))

X = X.float()
Y = Y.long()

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters())

for epoch in range(2000):
    optimizer.zero_grad()
    
    Y_hat = net(X)
    loss = criterion(Y_hat, Y)
    loss.backward()
    optimizer.step()
    
    if epoch % 100 == 0:
        print('epoch ' + str(epoch) + ' loss: ' + str(loss.item()))
        print('Accuracy = ' + str(np.mean(get_scores(Y, Y_hat))))
        
print('Done')

Net(
  (conv1): Conv1d(12, 32, kernel_size=(1,), stride=(1,))
  (fc1): Linear(in_features=32, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=24, bias=True)
)
epoch 0 loss: 3.1777231693267822
Accuracy = 0.12203125
epoch 100 loss: 3.024681568145752
Accuracy = 0.32593750000000005
epoch 200 loss: 2.808978319168091
Accuracy = 0.5514062500000001
epoch 300 loss: 2.7656826972961426
Accuracy = 0.58078125
epoch 400 loss: 2.7492055892944336
Accuracy = 0.5865625
epoch 500 loss: 2.7365546226501465
Accuracy = 0.5932812500000001
epoch 600 loss: 2.729646682739258
Accuracy = 0.59359375
epoch 700 loss: 2.7256698608398438
Accuracy = 0.593125
epoch 800 loss: 2.723574638366699
Accuracy = 0.5939062500000001
epoch 900 loss: 2.7223732471466064
Accuracy = 0.5946875
epoch 1000 loss: 2.721637487411499
Accuracy = 0.5935937499999999
epoch 1100 loss: 2.72031569480896
Accuracy = 0.59578125
epoch 1200 loss: 2.719661235809326
Accuracy = 0.595
epoch 1300 loss: 2.719283103942871
Accuracy = 0.

In [6]:
# Train accuracy
np.mean(get_scores(Y, net(X)))

0.59546875

In [7]:
# Test accuracy
X, Y = fileio.load_chroma_splits(range(20,27))
X = torch.transpose(torch.from_numpy(X), 0, 2)
Y = torch.from_numpy(keys.generate_one_hot_matrix(Y))

X = X.float()
Y = Y.long()

np.mean(get_scores(Y, net(X)))

0.4888324873096447